### Initialization
* Check whether the runtime is host or local.
* Mount Google Drive when using the host runtime.

In [0]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
  runtime = "host"
except:
  runtime = "local"

### Parameters

In [0]:
#@title Parameters
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`seed`|The random seed|
seed = 3984 #@param {type: "number"}

#@markdown ### `deep-coder` Repositories
#@markdown |Name            |Description|
#@markdown |:---            |:---|
#@markdown |`repository_url`|The URL of `deep-coder` git repository (enabled only in the host runtime)|
#@markdown |`branch_name`   |The branch name (enabled only in the host runtime)|
repository_url = "https://github.com/HiroakiMikami/deep-coder" #@param {type: "string"}
branch_name = "master" #@param {type: "string"}

#@markdown ### Filepathes
#@markdown |Name              |Description|
#@markdown |:---              |:---|
#@markdown |`model_shape_path`|The file path of the model shape.|
#@markdown |`model_path`      |The file path of the model parameters.|
model_shape_path = "out/model-shape.pickle" #@param {type: "string"}
model_path = "out/model.npz" #@param {type: "string"}



### Setup
* Fix the random seed
* Download the codebase (when using the host runtime)
  1. Clone git repository and move to the specified branch
  2. Initialize submodule
  3. Build the `search` tool
  4. Install chainer and cupy

In [0]:
import numpy as np
import random

SEED_MAX = 2**32 - 1

root_rng = np.random.RandomState(seed)
random.seed(root_rng.randint(SEED_MAX))
np.random.seed(root_rng.randint(SEED_MAX))

In [0]:
if runtime == "host":
  %cd /content
  !rm -rf deep-coder
  ![ ! -e deep-coder ] && git clone $repository_url deep-coder
  %cd deep-coder
  !git checkout origin/$branch_name
  !git submodule init
  !git submodule update
  !make -C DeepCoder_Utils/enumerative-search -j `nproc`
  !curl https://colab.chainer.org/install | sh -

### Load Model
* Load model

In [0]:
import pickle
import chainer as ch
from src.model import ModelShapeParameters, Predictor

# Load model
with open(model_shape_path, "rb") as f:
    model_shape: ModelShapeParameters = pickle.load(f)
predictor = Predictor(model_shape)
ch.serializers.load_npz(model_path, predictor)

### Visualize Model
* Show integer embeddings

In [0]:
#@markdown ### Visualization Parameters
#@markdown |Name    |Description|
#@markdown |:---    |:---|
#@markdown |`width` |The width of the matplotlib plot|
#@markdown |`height`|The height of the matplotlib plot|
#@markdown ---
width = 12 #@param {type: "slider", min: 1, max: 48, step: 1}
height = 12 #@param {type: "slider", min: 1, max: 48, step: 1}


In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(width, height))

embed = list(predictor.children())[0]._embed_integer
axis_0, axis_1 = np.random.choice(model_shape.n_embed, 2, replace=False)
for i in range(-model_shape.dataset_metadata.value_range, model_shape.dataset_metadata.value_range):
    e = embed(np.array([i + model_shape.dataset_metadata.value_range]))
    x = e.array[0, axis_0]
    y = e.array[0, axis_1]

    if i == 0:
        color = "b"
    elif i > 0:
        color = "g"
    else:
        color = "r"

    if i % 2 == 0:
        shape = "s"
    else:
        shape = "^"

    plt.plot(x, y, "{}{}".format(color, shape))
    if abs(i) < 10 or abs(i) > 253:
        plt.annotate("{}".format(i), xy=(x, y))

e = embed(np.array([2 * model_shape.dataset_metadata.value_range]))
x = e.array[0, axis_0]
y = e.array[0, axis_1]
plt.plot(x, y, "x")
plt.annotate("Null", xy=(x, y))
